In [ ]:
from kaskada.api.session import LocalBuilder
import kaskada.table
import time

In [ ]:
import os
os.environ['KASKADA_DISABLE_DOWNLOAD'] = "true" # only required here since Github rate limits

# Local Session (Next Iteration)

The previous version of the local session was designed to simply spin up the processes as subprocesses without much consideration for the use cases. As we've received feedback, the local session was due for a much needed next iteration. The goal of this notebook is to demonstrate the following new features:

* Session Stop
* Reuse an Existing Session
* Persist by Default
* Health Checks
* Auto recovery

This work is intended to be demonstrated on 5/23/2023.

## Session Stop

Previously, we utilized the garbage collector to stop the session but there are use cases where you want to stop the existing session (e.g. explicit resource management). The next iteration introduces a `stop()` method on an existing session and will attempt to stop all resources spawned by the session. This method is synchronous and waits for all resources to terminate.

```python
session = LocalBuilder().build()
...
session.stop()

```

In [ ]:
session = LocalBuilder().build()

In [ ]:
kaskada.table.list_tables()

## Reuse an Existing Session

If a local session already exists and is healthy, re-running the same session creation cell will result in the same session. This is done automatically with health checks. To use a different session, you will need to stop the existing one.

In [ ]:
session = LocalBuilder().build() # a session already exists from above. This is a major win.

## Persist by Default
The previous implementation ran with an in-memory database to manage resources. This means that between runs of the Manager, the current state of resources is flushed. By default, the new version will utilize a local sqlite database. This is configured to be `~/.cache/kaskada/data/wren.db`.

```python
session = LocalBuilder().build()
```

To restore the original functionality of an in-memory store, use the `in_memory` method on the **LocalBuilder**.
```python
session = LocalBuilder().in_memory(True).build()
```

To specify the location of the sqlite database, use the `database_path` method on the **LocalBuilder**.
```python
session = LocalBuilder().database_path("~/temp/database_v1.db").build()
```

In [ ]:
!rm ~/.cache/kaskada/data/wren.db

In [ ]:
session.stop()
session = LocalBuilder().build()

In [ ]:
kaskada.table.create_table(
  table_name = "test_table",
  entity_key_column_name = "entity_column",
  time_column_name = "time_column",
)

In [ ]:
kaskada.table.list_tables()

In [ ]:
session.stop()                   # stop the existing session
session = LocalBuilder().build() # create a new session

In [ ]:
kaskada.table.list_tables()

In [ ]:
# Previous implementation
session.stop()
session = LocalBuilder().in_memory(True).build()

In [ ]:
kaskada.table.list_tables()

In [ ]:
# Explicit use a local database
session.stop()
session = LocalBuilder().database_path("./my_awkward_database.db").build()

In [ ]:
kaskada.table.list_tables()

## Health Checks

Previously, we did not verify the existence or readiness of a local service session after spawning the subprocesses. This caused many customers difficulty in understanding why can the client not connect or understanding the current state of their Kaskada service. In this version, we built in the following health checks:
* Health Check Client - A client will now attempt to get the health of the service prior to reporting connected.
* Health Check Servicer - A thread-safe poller on the client that monitors the health of the client.
* Local Session Checks - A local service will now attempt to get the health of all services (manager and engine).
* Local Session Water - A thread-safe poller to monitor the status of the local running session.

In [ ]:
session.stop()
session = LocalBuilder().build()

In [ ]:
from kaskada.health.health_check_client import HealthCheckClientFactory

health_client_factory = HealthCheckClientFactory()
health_client_factory.get_client(
    kaskada.client.KASKADA_MANAGER_DEFAULT_HEALTH_CHECK_ENDPOINT, 
    kaskada.client.KASKADA_IS_SECURE
).check()

In [ ]:
from kaskada.health.health_check_client import HealthCheckClient

health_client_factory.get_client(
    kaskada.client.KASKADA_ENGINE_DEFAULT_HEALTH_CHECK_ENDPOINT, 
    kaskada.client.KASKADA_IS_SECURE
).check()

In [ ]:
session.stop()

In [ ]:
health_client_factory.get_client(
    kaskada.client.KASKADA_ENGINE_DEFAULT_HEALTH_CHECK_ENDPOINT, 
    kaskada.client.KASKADA_IS_SECURE
).check()

In [ ]:
# start everything up again
session = LocalBuilder().build()

from kaskada.health.health_check_servicer import HealthCheckServicer

# A servicer watches multiple services
health_servicer = HealthCheckServicer()
health_servicer.add_service('manager', kaskada.client.KASKADA_MANAGER_DEFAULT_HEALTH_CHECK_ENDPOINT, kaskada.client.KASKADA_IS_SECURE)
print(f"Single Service: {health_servicer.check()}")
print(f"The manager: {health_servicer.get('manager')}")

In [ ]:
health_servicer.add_service('engine', kaskada.client.KASKADA_ENGINE_DEFAULT_HEALTH_CHECK_ENDPOINT, kaskada.client.KASKADA_IS_SECURE)
print(f"Multiple Service: {health_servicer.check()}")
print(f"The engine: {health_servicer.get('engine')}")

In [ ]:
health_servicer.check()

In [ ]:
session.stop()

## Auto Recovery

The previous implementation spawned off the Kaskada processes without much concern or monitoring of their status throughout the operation. The notebook environments may perform garbage collection or kernel freezes will automatically kill the spawned processes resulting in a very undesirable state for the service.

Moving forward, the LocalBuilder will now include an auto-recovery feature to restore the services and client to connectivity on failure. Since this is almost always the intended functionality, this is not a configurable feature.

In [ ]:
!rm ~/.cache/kaskada/data/wren.db

In [ ]:
session = LocalBuilder().build()

In [ ]:
kaskada.table.list_tables()

In [ ]:
# Create the table named transactions with the time and name column
kaskada.table.create_table('transactions', 'time', 'name')
# Load the data to the table
kaskada.table.load('transactions', 'dataset1.csv')

In [ ]:
%load_ext fenlmagic

In [ ]:
%%fenl
transactions

*Go to another terminal and kill 50051.*

In [ ]:
%%fenl
transactions

In [ ]:
# Disable auto recovery by setting keep_alive to False
session = LocalBuilder().keep_alive(False).build()